# Concatenate all X's and Y's(merge them all)

In [1]:
import numpy as np

## Load 50% overlap data

In [2]:

# Load your arrays
#X_train3 = np.load('/kaggle/input/x-train-50overlap/X_train3.npy')
#y_train1 = np.load('/kaggle/input/x-train-50overlap/y_train1.npy')
#y_train2 = np.load('/kaggle/input/x-train-50overlap/y_train2.npy')
#y_train3 = np.load('/kaggle/input/x-train-50overlap/y_train3.npy')
#X_test = np.load('/kaggle/input/x-test3/X_test3.npy')
#y_test = np.load('/kaggle/input/x-test3/y_test3.npy')
#print("Arrays loaded!")

## Load no overlap data

In [3]:

# Load your arrays
X_train1 = np.load('/kaggle/input/x-train-no-overlap/X_train1_no_overlap.npy')
X_train2 = np.load('/kaggle/input/x-train-no-overlap/X_train2_no_overlap.npy')
X_train3 = np.load('/kaggle/input/x-train-no-overlap/X_train3_no_overlap.npy')
y_train1 = np.load('/kaggle/input/x-train-no-overlap/y_train1_no_overlap.npy')
y_train2 = np.load('/kaggle/input/x-train-no-overlap/y_train2_no_overlap.npy')
y_train3 = np.load('/kaggle/input/x-train-no-overlap/y_train3_no_overlap.npy')
X_test = np.load('/kaggle/input/x-train-no-overlap/X_test3_no_overlap.npy')
y_test = np.load('/kaggle/input/x-train-no-overlap/y_test3_no_overlap.npy')
print("Arrays loaded!")

Arrays loaded!


In [4]:
import numpy as np

# Assuming you've loaded your three datasets as:
# X_train1, y_train1
# X_train2, y_train2  
# X_train3, y_train3

# Merge all X arrays
# I did not include X_train3 and y_train3 because i want to observe the difference it give when testing on the test3 bearing 3
X_combined = np.concatenate([X_train1,X_train2,X_train3,X_test], axis=0)

# Merge all y arrays
y_combined = np.concatenate([y_train1,y_train2,y_train3,y_test], axis=0)

print(f"Combined shape: X={X_combined.shape}, y={y_combined.shape}")
print(f"Total windows: {len(X_combined)}")
print(f"Label distribution: {np.unique(y_combined, return_counts=True)}")

Combined shape: X=(376780, 2048, 1), y=(376780,)
Total windows: 376780
Label distribution: (array([0., 1.]), array([356160,  20620]))


In [5]:
# ✅ ADD THIS - SHUFFLE THE WINDOWS
print("Shuffling windows...")
indices = np.arange(len(X_combined))
np.random.seed(44)  # Reproducibility
np.random.shuffle(indices)

X_combined = X_combined[indices]
y_combined = y_combined[indices]
print("Shuffle complete!")

# Check it worked
print(f"First 20 labels after shuffle: {y_combined[:20]}")
print(f"Label distribution still same: {np.unique(y_combined, return_counts=True)}")


Shuffling windows...
Shuffle complete!
First 20 labels after shuffle: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
Label distribution still same: (array([0., 1.]), array([356160,  20620]))


In [6]:
print("Saving combined data...")
np.save('X_train.npy', X_combined)
np.save('y_train.npy', y_combined)

Saving combined data...


In [7]:
# Free ALL RAM
del X_train1, X_train2, X_train3, y_train1, y_train2, y_train3
del X_combined, y_combined
import gc
gc.collect()
print("RAM freed!")

RAM freed!


In [8]:
from tensorflow.keras.utils import Sequence
import numpy as np

# Step 1: Define the generator class (you need this!)
class DiskDataGenerator(Sequence):
    def __init__(self, X_path, y_path, batch_size=32):
        self.X_path = X_path
        self.y_path = y_path
        self.batch_size = batch_size
        
        # Memory-map the files (doesn't load into RAM)
        self.X = np.load(X_path, mmap_mode='r')
        self.y = np.load(y_path, mmap_mode='r')
        self.n_samples = len(self.X)
        
    def __len__(self):
        return int(np.ceil(self.n_samples / self.batch_size))
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, self.n_samples)
        
        # Only load this batch into RAM
        batch_x = np.array(self.X[start_idx:end_idx])
        batch_y = np.array(self.y[start_idx:end_idx])
        
        return batch_x, batch_y

2025-10-20 17:31:15.941524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760981476.180126      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760981476.251821      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Now use the generator from before
train_gen = DiskDataGenerator('X_train.npy', 'y_train.npy', batch_size=32)


In [10]:
# Load with memory mapping
X_train = np.load('X_train.npy', mmap_mode='r')
y_train = np.load('y_train.npy', mmap_mode='r')

# Basic checks
print("=== DATA INSPECTION ===")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_train dtype: {X_train.dtype}")
print(f"y_train dtype: {y_train.dtype}")

# Check for NaN/Inf
print(f"\nNaN in X: {np.isnan(X_train[:1000]).any()}")
print(f"Inf in X: {np.isinf(X_train[:1000]).any()}")

# Check value ranges
print(f"\nX_train range: [{np.min(X_train[:1000])}, {np.max(X_train[:1000])}]")
print(f"X_train mean: {np.mean(X_train[:1000])}")
print(f"X_train std: {np.std(X_train[:1000])}")

# Check labels
print(f"\ny_train unique: {np.unique(y_train)}")
print(f"Class distribution: {np.bincount(y_train.astype(int))}")

# Look at a few samples
print(f"\nFirst 10 labels: {y_train[:10]}")
print(f"Random 10 labels: {y_train[np.random.choice(len(y_train), 10)]}")

=== DATA INSPECTION ===
X_train shape: (376780, 2048, 1)
y_train shape: (376780,)
X_train dtype: float64
y_train dtype: float64

NaN in X: False
Inf in X: False

X_train range: [-2.085, 1.819]
X_train mean: -0.03811142431640625
X_train std: 0.0994882488504484

y_train unique: [0. 1.]
Class distribution: [356160  20620]

First 10 labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Random 10 labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Training the 1D CNN

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
print("done")

done


In [12]:
model = models.Sequential([
    layers.Conv1D(64, 15, activation='relu', input_shape=(2048, 1)),
    layers.MaxPooling1D(2),
    layers.Dropout(0.3),
    
    layers.Conv1D(128, 10, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Dropout(0.3),
    
    layers.Conv1D(128, 5, activation='relu'),  # 3rd layer
    layers.MaxPooling1D(2),
    layers.Dropout(0.3),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),       # Dense layer before output
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
print("Done")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1760981490.529168      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760981490.529815      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Done


## Step 8: See Your Complete Architecture

In [13]:
print("\n" + "="*50)
print("YOUR COMPLETE CNN ARCHITECTURE:")
print("="*50)
model.summary()


YOUR COMPLETE CNN ARCHITECTURE:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 2034, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1017, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1017, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1008, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 504, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 504, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 500, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 250, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 250, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,048,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,213,249 (8.44 MB)

 Trainable params: 2,213,249 (8.44 MB)

 Non-trainable params: 0 (0.00 B)

## Now Let's Compile the Model

In [14]:
from tensorflow.keras import metrics

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', metrics.Recall(name='recall')]
)
print("done")


done


## Step 10: Train the Model

In [15]:
# add class weights
class_weight_dict = {0.0: 1.0, 1.0: 17.0}

# Train the CNN
print("Starting training...")
history = model.fit(
    train_gen,             # using the generator
    epochs=5,                 # Go through all data 20 times
    #class_weight=class_weight_dict,
    verbose=1                  # Show progress
)

print("Training completed!")

Starting training...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1760981494.685604      96 service.cc:148] XLA service 0x79592c0033c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760981494.686638      96 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760981494.686658      96 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760981495.078734      96 cuda_dnn.cc:529] Loaded cuDNN version 90300


    5/11775 ━━━━━━━━━━━━━━━━━━━━ 6:00 31ms/step - accuracy: 0.7024 - loss: 0.5278 - recall: 0.0000e+00

I0000 00:00:1760981499.584983      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11775/11775 ━━━━━━━━━━━━━━━━━━━━ 224s 18ms/step - accuracy: 0.9815 - loss: 0.0564 - recall: 0.7024
Epoch 2/5
11775/11775 ━━━━━━━━━━━━━━━━━━━━ 215s 18ms/step - accuracy: 0.9939 - loss: 0.0172 - recall: 0.9224
Epoch 3/5
11775/11775 ━━━━━━━━━━━━━━━━━━━━ 215s 18ms/step - accuracy: 0.9952 - loss: 0.0147 - recall: 0.9419
Epoch 4/5
11775/11775 ━━━━━━━━━━━━━━━━━━━━ 215s 18ms/step - accuracy: 0.9959 - loss: 0.0128 - recall: 0.9534
Epoch 5/5
11775/11775 ━━━━━━━━━━━━━━━━━━━━ 214s 18ms/step - accuracy: 0.9959 - loss: 0.0127 - recall: 0.9568
Training completed!


## Step 11: Evaluate the Results

In [16]:
print(f"Class distribution: {np.bincount(y_test.astype(int))}")

Class distribution: [55510  3070]


In [17]:
## Make predictions on test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Check what it's actually predicting
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print(f"\nPrediction breakdown:")
unique, counts = np.unique(y_pred, return_counts=True)
for cls, count in zip(unique, counts):
    percentage = (count / len(y_pred)) * 100
    print(f"Predicted class {cls}: {count} samples ({percentage:.1f}%)")

1831/1831 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     55510
         1.0       1.00      1.00      1.00      3070

    accuracy                           1.00     58580
   macro avg       1.00      1.00      1.00     58580
weighted avg       1.00      1.00      1.00     58580


Confusion Matrix:
[[55496    14]
 [    4  3066]]

Prediction breakdown:
Predicted class 0: 55500 samples (94.7%)
Predicted class 1: 3080 samples (5.3%)


# testing on the CWRU(0% overlap)

In [19]:
#Loading the data
X_CWRU_test = np.load('/kaggle/input/cwru-dataprep/X_CWRU_test_no_overlap.npy')
y_CWRU_test = np.load('/kaggle/input/cwru-dataprep/y_CWRU_test_no_overlap.npy')
print("done")

done


In [ ]:
# per windows normalization
#X_CWRU_test_scaled = (X_CWRU_test - np.mean(X_CWRU_test, axis=1, keepdims=True)) / np.std(X_CWRU_test, axis=1, keepdims=True)

In [20]:

# 1️⃣ Compute mean and std from training data only
mean = np.mean(X_train)
std = np.std(X_train)

# 2️⃣ Apply normalization using those stats
X_train_scaled = (X_train - mean) / std
X_CWRU_test_scaled = (X_CWRU_test - mean) / std

# 3️⃣ Quick verification
print("Train mean (≈0):", np.mean(X_train_scaled))
print("Train std (≈1):", np.std(X_train_scaled))
print("CWRU test mean (close to 0):", np.mean(X_CWRU_test_scaled))
print("CWRU test std (close to 1):", np.std(X_CWRU_test_scaled))


Train mean (≈0): 3.156854551404562e-16
Train std (≈1): 1.0000000000000007
CWRU test mean (close to 0): 0.5540476460839914
CWRU test std (close to 1): 4.661215505854279


In [21]:
print(f"Class distribution: {np.bincount(y_CWRU_test.astype(int))}")

Class distribution: [ 236 2133]


In [22]:
print(f"Class distribution: {np.bincount(y_train.astype(int))}")

Class distribution: [356160  20620]


In [24]:
## Make predictions on test set
y_pred_prob = model.predict(X_CWRU_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Check what it's actually predicting
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_CWRU_test, y_pred))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_CWRU_test, y_pred)
print(cm)

print(f"\nPrediction breakdown:")
unique, counts = np.unique(y_pred, return_counts=True)
for cls, count in zip(unique, counts):
    percentage = (count / len(y_pred)) * 100
    print(f"Predicted class {cls}: {count} samples ({percentage:.1f}%)")

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       236
           1       0.90      0.95      0.92      2133

    accuracy                           0.86      2369
   macro avg       0.45      0.48      0.46      2369
weighted avg       0.81      0.86      0.83      2369


Confusion Matrix:
[[   0  236]
 [ 105 2028]]

Prediction breakdown:
Predicted class 0: 105 samples (4.4%)
Predicted class 1: 2264 samples (95.6%)
